<a href="https://colab.research.google.com/github/jaewook94/wandb/blob/main/colabs/pytorch/PyTorch_Sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Hyperparameter Sweeps using W&B

Searching through high dimensional hyperparameter spaces to find the most performant model can get unwieldy very fast. Hyperparameter sweeps provide an organized and efficient way to conduct a battle royale of models and pick the most accurate model. They enable this by automatically searching through combinations of hyperparameter values (e.g. learning rate, batch size, number of hidden layers, optimizer type) to find the most optimal values.

In this tutorial we'll see how you can run sophisticated hyperparameter sweeps in 3 easy steps using Weights and Biases.

![](https://i.imgur.com/WVKkMWw.png)

## Sweeps: An Overview

Running a hyperparameter sweep with Weights & Biases is very easy. There are just 3 simple steps:

1. **Define the sweep:** we do this by creating a dictionary or a [YAML file](https://docs.wandb.com/library/sweeps/configuration) that specifies the parameters to search through, the search strategy, the optimization metric et all.

2. **Initialize the sweep:** with one line of code we initialize the sweep and pass in the dictionary of sweep configurations:
`sweep_id = wandb.sweep(sweep_config)`

3. **Run the sweep agent:** also accomplished with one line of code, we call wandb.agent() and pass the sweep_id to run, along with a function that defines your model architecture and trains it:
`wandb.agent(sweep_id, function=train)`

And voila! That's all there is to running a hyperparameter sweep! In the notebook below, we'll walk through these 3 steps in more detail.


We highly encourage you to fork this notebook, tweak the parameters, or try the model with your own dataset!

## Resources
- [Sweeps docs →](https://docs.wandb.com/library/sweeps)
- [Launching from the command line →](https://www.wandb.com/articles/hyperparameter-tuning-as-easy-as-1-2-3)


# Setup
Start out by installing the experiment tracking library and setting up your free W&B account:


*   **pip install wandb** – Install the W&B library
*   **import wandb** – Import the wandb library

In [ ]:
!pip install wandb
!wandb login

## 1. Define the Sweep

Weights & Biases sweeps give you powerful levers to configure your sweeps exactly how you want them, with just a few lines of code. The sweeps config can be defined as a dictionary or a [YAML file](https://docs.wandb.com/library/sweeps).

Let's walk through some of them together:
*   **Metric** – This is the metric the sweeps are attempting to optimize. Metrics can take a `name` (this metric should be logged by your training script) and a `goal` (maximize or minimize). 
*   **Search Strategy** – Specified using the 'method' variable. We support several different search strategies with sweeps. 
  *   **Grid Search** – Iterates over every combination of hyperparameter values.
  *   **Random Search** – Iterates over randomly chosen combinations of hyperparameter values.
  *   **Bayesian Search** – Creates a probabilistic model that maps hyperparameters to probability of a metric score, and chooses parameters with high probability of improving the metric. The objective of Bayesian optimization is to spend more time in picking the hyperparameter values, but in doing so trying out fewer hyperparameter values.
*   **Stopping Criteria** – The strategy for determining when to kill off poorly peforming runs, and try more combinations faster. We offer several custom scheduling algorithms like [HyperBand](https://arxiv.org/pdf/1603.06560.pdf) and Envelope.
*   **Parameters** – A dictionary containing the hyperparameter names, and discreet values, max and min values or distributions from which to pull their values to sweep over.

You can find a list of all configuration options [here](https://docs.wandb.com/library/sweeps/configuration).

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torch.nn as nn
import wandb
from torchvision import datasets, transforms

In [2]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'epochs': {
            'values': [2, 5, 10]
        },
        'batch_size': {
            'values': [256, 128, 64, 32]
        },
        'dropout': {
            'values': [0.3, 0.4, 0.5]
        },
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
        },
        'fc_layer_size':{
            'values':[128,256,512]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        },
    }
}

## 2. Initialize the Sweep

In [3]:
sweep_id = wandb.sweep(sweep_config, project="Pytorch-sweeps")

Create sweep with ID: xr60wnto
Sweep URL: https://wandb.ai/jaewooklee/Pytorch-sweeps/sweeps/xr60wnto


In [10]:
sweep_id

'xr60wnto'

In [5]:
def build_dataset(batch_size):
   transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
   dataset = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
   train_loader = torch.utils.data.DataLoader(dataset,batch_size=batch_size)

   return train_loader

In [8]:
a = build_dataset(30)
a

### Define Your Neural Network
Before we can run the sweep, let's define a function that creates and trains our neural network.

In the function below, we define a simple fully-connected neural network in Pytorch, and add the following lines of code to log models metrics, visualize performance and output and track our experiments easily:
*   **wandb.init()** – Initialize a new W&B run. Each run is single execution of the training script.
*   **wandb.config** – Save all your hyperparameters in a config object. This lets you use our app to sort and compare your runs by hyperparameter values.
*   **wandb.log()** – Logs custom objects – these can be images, videos, audio files, HTML, plots, point clouds etc. Here we use wandb.log to log images of Simpson characters overlaid with actual and predicted labels.

In [9]:

def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 5,
        'batch_size': 128,
        'learning_rate': 1e-3,
        'optimizer': 'adam',
        'fc_layer_size': 128,
        'dropout': 0.5,
    }
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    
    # Define the model architecture - This is a simplified version of the VGG19 architecture
    network = nn.Sequential(
     nn.Flatten(start_dim=1)
    ,nn.Linear(784, config.fc_layer_size)
    ,nn.ReLU()
    ,nn.Dropout(config.dropout)
    ,nn.Linear(config.fc_layer_size, 10)
    ,nn.LogSoftmax(dim=1)
     )
    train_loader = build_dataset(config.batch_size)
    # Set of Conv2D, Conv2D, MaxPooling2D layers with 32 and 64 filters

    # Define the optimizer
    if config.optimizer=='sgd':
      optimizer = optim.SGD(network.parameters(),lr=config.learning_rate, momentum=0.9)
    elif config.optimizer=='adam':
      optimizer = optim.Adam(network.parameters(),lr=config.learning_rate)

    network.train()
    network = network.to(device)
    for i in range(config.epochs):
      closs= 0
      for batch_idx, (data, target) in enumerate(train_loader):
          data, target = data.to(device), target.to(device)
          optimizer.zero_grad()
          output = network(data)
          loss = F.nll_loss(output, target)
          loss.backward()
          closs = closs + loss.item()
          optimizer.step()
          wandb.log({"batch loss":loss.item()})
      wandb.log({"loss":closs/config.batch_size}) 

## 3. Run the sweep agent

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: wlijeloy with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.4
wandb: 	epochs: 5
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 3e-05
wandb: 	optimizer: sgd
wandb: Currently logged in as: jaewooklee (use `wandb login --relogin` to force relogin)


batch loss,1.99279
_runtime,78
_timestamp,1627634377
_step,1179
loss,1.75437


batch loss,████▇▇▇▇▆▆▆▆▆▆▆▅▅▅▄▅▅▄▄▄▄▄▄▄▃▃▃▂▂▂▃▁▁▂▂▁
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▄▃▁


wandb: Agent Starting Run: 1v0yvekm with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.5
wandb: 	epochs: 5
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 3e-05
wandb: 	optimizer: adam


batch loss,0.10085
_runtime,123
_timestamp,1627634506
_step,9379
loss,16.29255


...

batch loss,█▄▃▃▃▂▃▃▃▃▃▂▁▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▁▂▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▂▁▁


wandb: Agent Starting Run: t2ywe8eh with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.4
wandb: 	epochs: 2
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 3e-05
wandb: 	optimizer: sgd


batch loss,1.91272
_runtime,46
_timestamp,1627638671
_step,939
loss,7.11853


batch loss,███▇█▇█▇▇▆▆▆▆▅▆▅▅▅▄▄▄▄▃▃▃▃▂▃▃▂▂▃▁▂▂▁▁▁▂▁
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁


wandb: Agent Starting Run: y80xjy7f with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd


# Visualize Sweeps Results

## Parallel coordinates plot
This plot maps hyperparameter values to model metrics. It’s useful for honing in on combinations of hyperparameters that led to the best model performance.

![](https://assets.website-files.com/5ac6b7f2924c652fd013a891/5e190366778ad831455f9af2_s_194708415DEC35F74A7691FF6810D3B14703D1EFE1672ED29000BA98171242A5_1578695138341_image.png)

## Hyperparameter Importance Plot
The hyperparameter importance plot surfaces which hyperparameters were the best predictors of, and highly correlated to desirable values for your metrics.

![](https://assets.website-files.com/5ac6b7f2924c652fd013a891/5e190367778ad820b35f9af5_s_194708415DEC35F74A7691FF6810D3B14703D1EFE1672ED29000BA98171242A5_1578695757573_image.png)

These visualizations can help you save both time and resources running expensive hyperparameter optimizations by honing in on the parameters (and value ranges) that are the most important, and thereby worthy of further exploration.

# Next step - Get your hands dirty with sweeps
We created a simple training script and [a few flavors of sweep configs](https://github.com/wandb/examples/tree/master/keras-cnn-fashion) for you to play with. We highly encourage you to give these a try. This repo also has examples to help you try more advanced sweep features like [Bayesian Hyperband](https://app.wandb.ai/wandb/examples-keras-cnn-fashion/sweeps/us0ifmrf?workspace=user-lavanyashukla), and [Hyperopt](https://app.wandb.ai/wandb/examples-keras-cnn-fashion/sweeps/xbs2wm5e?workspace=user-lavanyashukla).